## Configurações

In [0]:
%pip install faker

## Parâmetros de Configuração

- **NOME_APLICACAO_SPARK**: Nome identificador desta execução no ambiente Spark.
- **ANO_ESTATISTICA**: Ano base utilizado para buscar as estatísticas de volume de transações.
- **FATOR_ESCALA_VOLUME**: Fator de escala para reduzir o volume total de transações geradas. Por exemplo, `0.006` indica que será gerado apenas 0,6% do volume real, útil para criar amostras menores e mais rápidas de processar.
- **LIMITE_ABSOLUTO_TX**: Limite máximo absoluto de transações a serem geradas, evitando volumes excessivos.
- **LIMITE_MUNICIPIOS_PROCESSADOS**: Quantidade máxima de municípios para os quais serão gerados dados, priorizando aqueles com maior volume.
- **PROBABILIDADE_TRANSACAO_INTERMUNICIPAL**: Proporção de transações que terão como destino um município diferente do de origem (ex: 20%), tornando a simulação mais realista.

---

### Parâmetros de Fraude

- **PROBABILIDADE_FRAUDE_BASE**: Probabilidade padrão de uma transação ser fraudulenta (ex: 5%).
- **PROB_CONTA_ALTO_RISCO**: Probabilidade de uma conta ser classificada como "alto risco" ao ser criada (ex: 10%).
- **PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO**: Probabilidade de fraude quando o destino é uma conta de alto risco (ex: 60%).
- **PROBABILIDADE_FRAUDE_CHAVE_RECENTE**: Probabilidade de fraude quando o PIX é destinado a uma chave recém-cadastrada (ex: 40%), já que contas novas são frequentemente usadas em golpes.
- **DIAS_CHAVE_CONSIDERADA_RECENTE**: Número de dias para considerar uma chave como "recente" (ex: 7 dias).
- **MAX_DIAS_CADASTRO_CHAVE_RISCO**: Prazo máximo para contas de alto risco cadastrarem suas chaves PIX após a abertura (ex: 5 dias).

---

### Parâmetros Gerais

- **MULTIPLICADOR_MAGNITUDE_OUTLIER**: Multiplicador para definir o valor de uma transação "outlier" (ex: 25 vezes o valor base).
- **MULTIPLICADOR_MAGNITUDE_FRAUDE**: Multiplicador para definir o valor de uma transação fraudulenta (ex: 50 vezes o valor base), simulando tentativas de golpes de alto valor.
- **PROBABILIDADES_TIPO_FRAUDE**: Dicionário que define a chance de cada tipo de fraude ocorrer. Exemplo: `valor_atipico` tem 30% de chance de ser o tipo escolhido em caso de fraude.
- **PESO_CONTAS_POS_PIX**: Proporção de contas criadas após o lançamento do PIX (ex: 70%), refletindo o aumento da bancarização no período.

In [0]:
# =============================================================================
# CÉLULA 1: CONFIGURAÇÃO DE PARÂMETROS GLOBAIS (VERSÃO FINAL)
# =============================================================================
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType, StructField, StringType, DateType, DoubleType, IntegerType, TimestampType
)
from faker import Faker
import numpy as np
import pandas as pd
import random
import uuid
from datetime import timedelta, datetime
import calendar
from typing import Iterator
import functools

print("INFO: Definindo parâmetros globais para treinamento de IA (com estratégias avançadas)...")

# === PARÂMETROS DE ESCALA E VOLUME ===
ANO_ESTATISTICA = 2023
LIMITE_MUNICIPIOS_PROCESSADOS = 15 
FATOR_ESCALA_VOLUME = 0.00005 
TX_POR_CLIENTE_ESPERADO = 10.0 
PROBABILIDADE_TRANSACAO_INTERMUNICIPAL = 0.20 

# === PARÂMETROS DE RISCO E CONTA ===
PROB_CONTA_ALTO_RISCO = 0.03 
PESO_CONTAS_POS_PIX = 0.70 

# === PARÂMETROS DE FRAUDE (REALISTAS) ===
PROBABILIDADE_FRAUDE_BASE = 0.005 # (0.5%)
PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO = 0.60 
PROBABILIDADE_FRAUDE_CHAVE_RECENTE = 0.40      
MULTIPLICADOR_MAGNITUDE_FRAUDE = 30 
DIAS_CHAVE_CONSIDERADA_RECENTE = 7            
MAX_DIAS_CADASTRO_CHAVE_RISCO = 5 

PROBABILIDADES_TIPO_FRAUDE = {
    "valor_atipico": 0.30, "engenharia_social": 0.25, "tomada_de_conta": 0.15, 
    "triangulacao_conta_laranja": 0.15, "fraude_qr_code": 0.10, "ataque_de_frequencia": 0.05
}

# === PARÂMETROS DE ESTRATÉGIAS AVANÇADAS DE FRAUDE ===
PROBABILIDADE_ATAQUE_MADRUGADA = 0.70 
PROBABILIDADE_TESTE_CONTA = 0.30
PROBABILIDADE_ABAIXO_RADAR = 0.40
VALORES_LIMITE_RADAR = [499.90, 999.90, 1999.90, 4999.90]

## Inicialização do Ambiente e Esquemas

In [0]:
# =============================================================================
# CÉLULA 2: SETUP DO SPARK E DEFINIÇÃO DE SCHEMAS
# =============================================================================
NOME_APLICACAO_SPARK = "GeradorDadosPix_Final_v12.0_Otimizado"

fake = Faker('pt_BR')
np.random.seed(42)
random.seed(42)

spark = SparkSession.builder.appName(NOME_APLICACAO_SPARK).getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "200")

print("INFO: Definindo esquemas explícitos para UDFs e tabelas Delta...")

SCHEMA_PARES = StructType([
    StructField("id_conta_origem", StringType(), True),
    StructField("id_conta_destino", StringType(), True)
])

SCHEMA_CLIENTES_UDF = StructType([
    StructField("nome", StringType()),
    StructField("registro_nacional", StringType()),
    StructField("nascido_em", DateType())
])

SCHEMA_CONTAS_UDF = StructType([
    StructField("id", StringType()),
    StructField("saldo", DoubleType()),
    StructField("aberta_em", DateType()),
    StructField("agencia", StringType()),
    StructField("numero", StringType()),
    StructField("id_tipo_conta", IntegerType()),
    StructField("ispb_instituicao", StringType()),
    StructField("id_cliente", StringType()),
    StructField("is_high_risk", IntegerType())
])

SCHEMA_CHAVES_UDF = StructType([
    StructField("id", StringType()),
    StructField("chave", StringType()),
    StructField("id_tipo_chave", IntegerType()),
    StructField("cadastrada_em", DateType()),
    StructField("id_conta", StringType())
])

SCHEMA_TRANSACOES_UDF = StructType([
    StructField("id", StringType()),
    StructField("valor", DoubleType()),
    StructField("data", TimestampType()),
    StructField("mensagem", StringType()),
    StructField("id_conta_origem", StringType()),
    StructField("id_conta_destino", StringType()),
    StructField("id_tipo_iniciacao_pix", IntegerType()),
    StructField("id_finalidade_pix", IntegerType()),
    StructField("is_fraud", IntegerType()),
    StructField("fraud_type", StringType()),
    StructField("id_transacao_cadeia_pai", StringType())
])

SCHEMA_CLIENTES_FINAL = StructType([
    StructField("id", StringType(), False),
    StructField("nome", StringType()),
    StructField("id_natureza", IntegerType()),
    StructField("registro_nacional", StringType()),
    StructField("nascido_em", DateType()),
    StructField("estado_ibge", IntegerType()),
    StructField("municipio_ibge", IntegerType())
])

SCHEMA_CONTAS_FINAL = StructType(
    SCHEMA_CONTAS_UDF.fields + [
        StructField("estado_ibge", IntegerType()),
        StructField("municipio_ibge", IntegerType())
    ]
)

SCHEMA_CHAVES_PIX_FINAL = StructType(
    SCHEMA_CHAVES_UDF.fields + [
        StructField("estado_ibge", IntegerType()),
        StructField("municipio_ibge", IntegerType())
    ]
)

SCHEMA_TRANSACOES_FINAL = StructType(
    SCHEMA_TRANSACOES_UDF.fields + [
        StructField("estado_ibge", IntegerType())
    ]
)

config_geracao = {
    "PROBABILIDADE_FRAUDE_BASE": PROBABILIDADE_FRAUDE_BASE,
    "PROB_CONTA_ALTO_RISCO": PROB_CONTA_ALTO_RISCO,
    "PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO": PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO,
    "PROBABILIDADE_FRAUDE_CHAVE_RECENTE": PROBABILIDADE_FRAUDE_CHAVE_RECENTE,
    "DIAS_CHAVE_CONSIDERADA_RECENTE": DIAS_CHAVE_CONSIDERADA_RECENTE,
    "MAX_DIAS_CADASTRO_CHAVE_RISCO": MAX_DIAS_CADASTRO_CHAVE_RISCO,
    "MULTIPLICADOR_MAGNITUDE_FRAUDE": MULTIPLICADOR_MAGNITUDE_FRAUDE,
    "PROBABILIDADES_TIPO_FRAUDE": PROBABILIDADES_TIPO_FRAUDE,
    "PESO_CONTAS_POS_PIX": PESO_CONTAS_POS_PIX,
    
    # Adição dos novos parâmetros de estratégia
    "PROBABILIDADE_ATAQUE_MADRUGADA": PROBABILIDADE_ATAQUE_MADRUGADA,
    "PROBABILIDADE_TESTE_CONTA": PROBABILIDADE_TESTE_CONTA,
    "PROBABILIDADE_ABAIXO_RADAR": PROBABILIDADE_ABAIXO_RADAR,
    "VALORES_LIMITE_RADAR": VALORES_LIMITE_RADAR}

print("INFO: Carregando dados de dimensão e estatísticas...")
try:
    instituicoes_pd = spark.table("transacoes_db.copper.instituicoes").toPandas()
    tipos_conta_pd = spark.table("transacoes_db.copper.tipos_conta").toPandas()
    perfis_de_uso_dict = spark.table("transacoes_db.pix_baseline_metricas.perfil_de_usuarios").toPandas().to_dict('records')
    LISTA_ISPBS_LOCAL = instituicoes_pd['ispb'].tolist()
    LISTA_TIPOS_CONTA_LOCAL = tipos_conta_pd['id'].tolist()
    print("INFO: Dados auxiliares e estatísticas carregados com sucesso.")
except Exception as e:
    print(f"ERRO: Falha ao carregar dados de dimensão ou estatísticas. Erro: {e}"); raise e

In [0]:
%sql
CREATE OR REPLACE TABLE transacoes_db.pix_baseline_metricas.volumes_anuais_por_municipio
COMMENT 'Tabela agregada com os volumes totais anuais por município para acelerar o pipeline de geração de dados.'
AS
SELECT
  ano AS Ano,
  Municipio_Ibge AS cod_ibge_municipio,
  Municipio AS municipio_nome,
  
  -- ALTERAÇÃO APLICADA AQUI: Derivando o código do estado a partir do código do município
  CAST(SUBSTRING(CAST(Municipio_Ibge AS STRING), 1, 2) AS INT) AS cod_ibge_estado,
  
  -- Calculando o total de transações de pagadores
  SUM(total_tx_pf_pagador + total_tx_pj_pagador) AS volume_pagador_anual,
  
  -- Mantendo os nomes das colunas de origem para o cálculo do número de clientes
  SUM(total_tx_pf_pagador) AS total_pf_anual,
  SUM(total_tx_pj_pagador) AS total_pj_anual
FROM
  transacoes_db.pix_baseline_metricas.relacao_pagadores_recebedores
GROUP BY
  ano,
  Municipio_Ibge,
  Municipio,
  -- Agrupando também pela coluna derivada
  CAST(SUBSTRING(CAST(Municipio_Ibge AS STRING), 1, 2) AS INT);

In [0]:
%sql
OPTIMIZE transacoes_db.pix_baseline_metricas.relacao_pagadores_recebedores;


OPTIMIZE transacoes_db.pix_baseline_metricas.relacao_pagadores_recebedores
ZORDER BY (ano, Municipio_Ibge);

## Funções de Geração e Salvamento

As funções abaixo trabalham em conjunto para criar a população e as transações de um município:

---

### 1. `_gerar_clientes`, `_gerar_contas`, `_gerar_chaves_pix`

- **_gerar_clientes**  
  Cria o número especificado de Pessoas Físicas (PF) e Jurídicas (PJ), gerando IDs, nomes, CPFs/CNPJs e datas de nascimento/fundação.

- **_gerar_contas**  
  Para cada cliente, cria um número aleatório de contas bancárias.

- **_gerar_chaves_pix**  
  Para cada conta, gera uma chave PIX (CPF, e-mail, telefone, etc.), garantindo que a data de cadastro da chave seja sempre posterior à data de abertura da conta.

---

### 2. `_gerar_detalhes_transacao_python_vetorizado`

Função responsável por criar os detalhes de cada transação:

- **Data da Transação:**  
  Seleciona uma data e hora aleatória dentro do mês de referência.

- **Lógica de Fraude:**  
  Define a probabilidade de fraude com base em regras, como se a conta destino é de risco ou se a chave é recente.

- **Valor por Tipo de Conta:**  
  Gera um valor monetário condizente com o tipo de transação (ex: salário, transferência para poupança, etc.).

- **Multiplicadores de Fraude e Outlier:**  
  Aumenta drasticamente o valor da transação se ela for fraudulenta ou um outlier.

- **Fraudes em Cadeia:**  
  Para certos tipos de fraude, divide o valor inicial em várias subtransações menores, simulando lavagem de dinheiro.

---

### 3. `gerar_transacoes`

Orquestra a geração das transações para um município em um determinado mês:

- **Divisão do Volume:**  
  Separa o total de transações em locais e intermunicipais.

- **Geração de Pares Locais:**  
  Sorteia aleatoriamente pares de contas (origem e destino) do mesmo município.

- **Geração de Pares Intermunicipais:**  
  Sorteia contas de origem do município atual e contas de destino de outros municípios.

- **União e Enriquecimento:**  
  Junta os dois conjuntos de pares e busca as informações necessárias para gerar os detalhes de cada transação.

In [0]:
# =============================================================================
# CÉLULA 3: FUNÇÕES DE GERAÇÃO DE DADOS (VERSÃO FINAL COMPLETA)
# =============================================================================

# --- Funções de Geração de População ---

@F.pandas_udf(SCHEMA_CLIENTES_UDF)
def _gerar_detalhes_cliente_udf(id_natureza: pd.Series) -> pd.DataFrame:
    local_fake = Faker('pt_BR')
    n = len(id_natureza)
    nomes_pf = [local_fake.name() for _ in range(n)]; nomes_pj = [local_fake.company() for _ in range(n)]
    nomes = np.where(id_natureza == 1, nomes_pf, nomes_pj)
    registros_pf = [local_fake.cpf() for _ in range(n)]; registros_pj = [local_fake.cnpj() for _ in range(n)]
    registros = np.where(id_natureza == 1, registros_pf, registros_pj)
    nascimentos = [
        local_fake.date_of_birth(minimum_age=18, maximum_age=80) if nat == 1 
        else local_fake.date_between(start_date='-20y', end_date='-1y')
        for nat in id_natureza
    ]
    return pd.DataFrame({"nome": nomes, "registro_nacional": registros, "nascido_em": nascimentos})

def _gerar_clientes(num_pf: int, num_pj: int, estado_ibge: int, municipio_ibge: int) -> DataFrame:
    df_base_pf = spark.range(num_pf).withColumn("id_natureza", F.lit(1)); df_base_pj = spark.range(num_pj).withColumn("id_natureza", F.lit(2))
    df_base_clientes = df_base_pf.union(df_base_pj)
    return (df_base_clientes.withColumn("id", F.expr("uuid()"))
        .withColumn("detalhes", _gerar_detalhes_cliente_udf(F.col("id_natureza")))
        .select("id", F.col("detalhes.nome").alias("nome"), "id_natureza",
                F.col("detalhes.registro_nacional").alias("registro_nacional"),
                F.col("detalhes.nascido_em").alias("nascido_em"),
                F.lit(estado_ibge).alias("estado_ibge"), F.lit(municipio_ibge).alias("municipio_ibge")))

def _gerar_detalhes_conta_python(iterator: Iterator[pd.DataFrame], config: dict, tipos_conta: list, ispbs: list, ano_estatistica: int) -> Iterator[pd.DataFrame]:
    local_fake = Faker('pt_BR'); data_limite_abertura = datetime(ano_estatistica, 1, 1).date()
    for lote in iterator:
        resultados = []
        for row in lote.itertuples(index=False):
            is_high_risk = 1 if random.random() < config['PROB_CONTA_ALTO_RISCO'] else 0
            if is_high_risk == 1:
                start_date_relativa = timedelta(days=180); data_inicio_recente = data_limite_abertura - start_date_relativa
                aberta_em = local_fake.date_between(start_date=data_inicio_recente, end_date=data_limite_abertura)
            else: aberta_em = local_fake.date_between(start_date='-10y', end_date=data_limite_abertura)
            if row.id_natureza == 1: saldo = round(np.random.lognormal(mean=6, sigma=1.5), 2); tipo_conta = random.choice(tipos_conta)
            else: saldo = round(np.random.lognormal(mean=9, sigma=1.8), 2); tipo_conta = random.choice([c for c in tipos_conta if c in [1, 3]])
            resultados.append({"id": str(uuid.uuid4()), "saldo": saldo, "aberta_em": aberta_em, "agencia": local_fake.numerify('####'), 
                               "numero": local_fake.numerify('#####-#'), "id_tipo_conta": tipo_conta, "ispb_instituicao": random.choice(ispbs),
                               "id_cliente": row.id_cliente, "is_high_risk": is_high_risk})
        yield pd.DataFrame(resultados)

def _gerar_contas(df_clientes: DataFrame, estado_ibge: int, municipio_ibge: int) -> DataFrame:
    df_clientes_com_num_contas = df_clientes.withColumn("num_contas", F.when(F.col("id_natureza") == 1, F.floor(F.rand() * 2) + 1).otherwise(F.floor(F.rand() * 5) + 1))
    df_contas_base = df_clientes_com_num_contas.select(F.col("id").alias("id_cliente"), "id_natureza", F.explode(F.sequence(F.lit(1), F.col("num_contas"))))
    gerador_com_contexto = functools.partial(_gerar_detalhes_conta_python, config=config_geracao, tipos_conta=LISTA_TIPOS_CONTA_LOCAL, ispbs=LISTA_ISPBS_LOCAL, ano_estatistica=ANO_ESTATISTICA)
    return (df_contas_base.mapInPandas(gerador_com_contexto, schema=SCHEMA_CONTAS_UDF)
            .withColumn("estado_ibge", F.lit(estado_ibge)).withColumn("municipio_ibge", F.lit(municipio_ibge)))

def _gerar_detalhes_chave_udf(iterator: Iterator[pd.DataFrame], config: dict) -> Iterator[pd.DataFrame]:
    local_fake = Faker('pt_BR')
    for lote in iterator:
        resultados = [];
        for row in lote.itertuples(index=False):
            try:
                data_abertura_obj = pd.to_datetime(row.aberta_em).date()
                dias_para_cadastrar = random.randint(1, config['MAX_DIAS_CADASTRO_CHAVE_RISCO']) if hasattr(row, 'is_high_risk') and row.is_high_risk == 1 else random.randint(1, 90)
                cadastrada_em = data_abertura_obj + timedelta(days=dias_para_cadastrar)
                tipos_possiveis = {1: row.registro_nacional, 2: local_fake.email(), 3: local_fake.phone_number(), 4: str(uuid.uuid4())} if row.id_natureza == 1 else {5: row.registro_nacional, 2: local_fake.company_email(), 4: str(uuid.uuid4())}
                tipo_chave = random.choice(list(tipos_possiveis.keys()))
                resultados.append({"id": str(uuid.uuid4()), "chave": tipos_possiveis[tipo_chave], "id_tipo_chave": tipo_chave, "cadastrada_em": cadastrada_em, "id_conta": row.id_conta})
            except Exception as e: print(f"ERRO NA GERAÇÃO DE CHAVES: {e}, DADOS: {row}")
        if resultados: yield pd.DataFrame(resultados)

def _gerar_chaves_pix(df_contas_completo: DataFrame, df_clientes_completo: DataFrame, estado_ibge: int, municipio_ibge: int) -> DataFrame:
    df_contas_com_cliente = df_contas_completo.join(df_clientes_completo, df_contas_completo.id_cliente == df_clientes_completo.id, "inner").select(df_contas_completo.id.alias("id_conta"), "id_natureza", "registro_nacional", "aberta_em", df_contas_completo.is_high_risk)
    gerador_com_contexto = functools.partial(_gerar_detalhes_chave_udf, config=config_geracao)
    return (df_contas_com_cliente.mapInPandas(gerador_com_contexto, schema=SCHEMA_CHAVES_UDF)
            .withColumn("estado_ibge", F.lit(estado_ibge)).withColumn("municipio_ibge", F.lit(municipio_ibge)))

# --- Funções de Geração de Transações (com Lógica Avançada de Fraude) ---
def _obter_params_tempo(ano: int, mes: int) -> tuple:
    num_dias = calendar.monthrange(ano, mes)[1]; primeiro_dia = datetime(ano, mes, 1)
    ultimo_dia = datetime(ano, mes, num_dias, 23, 59, 59)
    return primeiro_dia, (ultimo_dia - primeiro_dia).total_seconds()

def _aplicar_horario_suspeito(data_transacao, config):
    if random.random() < config.get('PROBABILIDADE_ATAQUE_MADRUGADA', 0.70):
        return data_transacao.replace(hour=random.randint(1, 4), minute=random.randint(0, 59))
    return data_transacao

def _gerar_detalhes_transacao_python_vetorizado(iterator: Iterator[pd.DataFrame], ano: int, mes: int, config: dict, perfis_uso: list) -> Iterator[pd.DataFrame]:
    primeiro_dia, delta_segundos = _obter_params_tempo(ano, mes)
    colunas_finais_transacoes = ["id", "valor", "data", "mensagem", "id_conta_origem", "id_conta_destino", "id_tipo_iniciacao_pix", "id_finalidade_pix", "is_fraud", "fraud_type", "id_transacao_cadeia_pai"]
    PROBS_PROFUNDIDADE = [0.35, 0.60, 0.05]; PROB_RUIDO = 0.25; MENSAGENS_RUIDO = ["Pagamento de Boleto", "Lanchonete", "Uber", "Recarga de Celular"]
    for lote in iterator:
        n = len(lote)
        if n == 0: continue
        lote['data'] = primeiro_dia + pd.to_timedelta(np.random.uniform(0, delta_segundos, n), unit='s')
        lote['chave_destino_cadastrada_em'] = pd.to_datetime(lote['chave_destino_cadastrada_em']); delta_dias = (lote['data'] - lote['chave_destino_cadastrada_em']).dt.days
        lote['is_high_risk'] = lote['is_high_risk'].fillna(0).astype(int)
        prob_fraude_dinamica = np.select([lote['is_high_risk'] == 1, (delta_dias >= 0) & (delta_dias <= config['DIAS_CHAVE_CONSIDERADA_RECENTE'])], [config['PROBABILIDADE_FRAUDE_CONTA_DESTINO_RISCO'], config['PROBABILIDADE_FRAUDE_CHAVE_RECENTE']], default=config['PROBABILIDADE_FRAUDE_BASE'])
        lote['is_fraud'] = (np.random.rand(n) < prob_fraude_dinamica).astype(int)
        multiplicadores = np.select([lote['is_fraud'] == 1, (~(lote['is_fraud'] == 1)) & (np.random.rand(n) < 0.04)], [config['MULTIPLICADOR_MAGNITUDE_FRAUDE'], 2.5], default=1.0)
        valores_calculados = np.maximum(0.01, np.random.lognormal(mean=np.log(150), sigma=0.8, size=n) * multiplicadores).round(2)
        condicao_abaixo_radar = (lote['is_fraud'] == 1) & (np.random.rand(n) < config.get('PROBABILIDADE_ABAIXO_RADAR', 0.4))
        lote['valor'] = np.where(condicao_abaixo_radar, np.random.choice(config.get('VALORES_LIMITE_RADAR', [999.90]), n), valores_calculados)
        probs_tipo_fraude = config.get('PROBABILIDADES_TIPO_FRAUDE'); tipos_fraude = np.random.choice(list(probs_tipo_fraude.keys()), n, p=list(probs_tipo_fraude.values()))
        lote['fraud_type'] = np.where(lote['is_fraud'] == 1, tipos_fraude, None)
        lote['id'] = [str(uuid.uuid4()) for _ in range(n)]; lote['mensagem'] = "Pagamento via Pix"; lote['id_tipo_iniciacao_pix'] = np.random.randint(1, 4, n); lote['id_finalidade_pix'] = np.random.randint(1, 5, n); lote['id_transacao_cadeia_pai'] = None
        lote_final = lote.drop(columns=[col for col in ['chave_destino_cadastrada_em', 'is_high_risk', 'id_tipo_conta_origem', 'id_tipo_conta_destino'] if col in lote.columns])
        resultados_finais = []
        for row in lote_final.itertuples(index=False):
            if hasattr(row, 'is_fraud') and row.is_fraud and row.fraud_type in ["triangulacao_conta_laranja", "ataque_de_frequencia"]:
                if random.random() < config.get('PROBABILIDADE_TESTE_CONTA', 0.3):
                    resultados_finais.append({"id": str(uuid.uuid4()), "valor": round(random.uniform(0.01, 1.00), 2), "data": row.data - timedelta(minutes=random.randint(1, 5)), "mensagem": "Teste", "id_conta_origem": row.id_conta_origem, "id_conta_destino": row.id_conta_destino, "id_tipo_iniciacao_pix": 1, "id_finalidade_pix": 1, "is_fraud": 0, "fraud_type": None, "id_transacao_cadeia_pai": None})
                row_dict = row._asdict(); row_dict['data'] = _aplicar_horario_suspeito(row_dict['data'], config)
                profundidade_alvo = np.random.choice([2, 3, 4], p=PROBS_PROFUNDIDADE)
                id_fraude_raiz = row_dict['id']; resultados_finais.append(row_dict)
                contas_nivel_anterior = {row_dict['id_conta_destino']: row_dict['valor']}; id_pai_nivel_anterior = {row_dict['id_conta_destino']: id_fraude_raiz}; data_nivel_anterior = {row_dict['id_conta_destino']: row_dict['data']}
                for nivel_atual in range(2, profundidade_alvo + 1):
                    contas_proximo_nivel = {}; id_pai_proximo_nivel = {}; data_proximo_nivel = {}
                    if not contas_nivel_anterior: break
                    for conta_origem, valor_origem in contas_nivel_anterior.items():
                        num_subs = random.randint(2, 7); valores_divididos = np.random.dirichlet(np.ones(num_subs)) * valor_origem
                        for k in range(num_subs):
                            id_transacao_filha = str(uuid.uuid4()); id_conta_destino_filha = str(uuid.uuid4()); segundos_offset = random.uniform(60 * (nivel_atual-1), 3600 * (nivel_atual-1))
                            data_transacao_filha = data_nivel_anterior[conta_origem] + timedelta(seconds=segundos_offset); data_transacao_filha = _aplicar_horario_suspeito(data_transacao_filha, config)
                            transacao_filha = {"id": id_transacao_filha, "valor": round(max(0.01, valores_divididos[k]), 2), "data": data_transacao_filha, "mensagem": f"Dispersão N{nivel_atual} (Parte {k+1}/{num_subs})", "id_conta_origem": conta_origem, "id_conta_destino": id_conta_destino_filha, "id_tipo_iniciacao_pix": random.randint(1, 3), "id_finalidade_pix": random.randint(1, 4), "is_fraud": 1, "fraud_type": row.fraud_type, "id_transacao_cadeia_pai": id_pai_nivel_anterior[conta_origem]}
                            resultados_finais.append(transacao_filha)
                            contas_proximo_nivel[id_conta_destino_filha] = transacao_filha["valor"]; id_pai_proximo_nivel[id_conta_destino_filha] = id_transacao_filha; data_proximo_nivel[id_conta_destino_filha] = data_transacao_filha
                            if random.random() < PROB_RUIDO:
                                for _ in range(random.randint(1, 3)):
                                    segundos_offset_ruido = random.uniform(10, 3600)
                                    resultados_finais.append({"id": str(uuid.uuid4()), "valor": round(random.uniform(7.5, 75.0), 2), "data": data_transacao_filha + timedelta(seconds=segundos_offset_ruido), "mensagem": random.choice(MENSAGENS_RUIDO), "id_conta_origem": id_conta_destino_filha, "id_conta_destino": str(uuid.uuid4()), "id_tipo_iniciacao_pix": 1, "id_finalidade_pix": 1, "is_fraud": 0, "fraud_type": None, "id_transacao_cadeia_pai": None})
                    contas_nivel_anterior = contas_proximo_nivel; id_pai_nivel_anterior = id_pai_proximo_nivel; data_nivel_anterior = data_proximo_nivel
            else: resultados_finais.append(row._asdict())
        if resultados_finais: df_final = pd.DataFrame(resultados_finais); yield df_final[colunas_finais_transacoes]

def gerar_transacoes(
    df_contas_local: DataFrame, df_chaves_recentes_local: DataFrame, num_contas_local: int,
    volume_total: int, estado_ibge: int, municipio_ibge: int, ano: int, mes: int, 
    municipios_processados: list
) -> DataFrame:
    
    volume_intermunicipal = int(volume_total * PROBABILIDADE_TRANSACAO_INTERMUNICIPAL); volume_local = volume_total - volume_intermunicipal
    df_pares_locais = spark.createDataFrame([], SCHEMA_PARES); df_pares_intermunicipais = spark.createDataFrame([], SCHEMA_PARES)
    if volume_local > 0 and num_contas_local > 1:
        contas_ids_locais = df_contas_local.select("id"); num_contas_locais = num_contas_local
        df_pares_indices_locais = spark.range(volume_local).repartition(100).withColumn("idx_origem", (F.rand() * num_contas_locais).cast("long")).withColumn("idx_destino", (F.rand() * num_contas_locais).cast("long")).filter("idx_origem != idx_destino")
        contas_numeradas_locais = contas_ids_locais.withColumn("idx", F.row_number().over(Window.orderBy("id")) - 1)
        df_pares_locais = df_pares_indices_locais.join(contas_numeradas_locais.alias("orig"), df_pares_indices_locais.idx_origem == F.col("orig.idx")).join(contas_numeradas_locais.alias("dest"), df_pares_indices_locais.idx_destino == F.col("dest.idx")).select(F.col("orig.id").alias("id_conta_origem"), F.col("dest.id").alias("id_conta_destino"))
    outros_municipios = [m for m in municipios_processados if m != municipio_ibge]
    if volume_intermunicipal > 0 and outros_municipios:
        municipio_alvo = random.choice(outros_municipios)
        contas_origem = df_contas_local.select("id").orderBy(F.rand()).limit(volume_intermunicipal).withColumnRenamed("id", "id_conta_origem").withColumn("join_key", F.monotonically_increasing_id())
        contas_destino_externas = spark.table("transacoes_db.copper.contas").filter(F.col("municipio_ibge") == municipio_alvo).select("id").orderBy(F.rand()).limit(volume_intermunicipal).withColumnRenamed("id", "id_conta_destino").withColumn("join_key", F.monotonically_increasing_id())
        df_pares_intermunicipais = contas_origem.join(contas_destino_externas, "join_key").select("id_conta_origem", "id_conta_destino")
    df_pares_total = df_pares_locais.union(df_pares_intermunicipais)
    if df_pares_total.isEmpty(): return spark.createDataFrame([], SCHEMA_TRANSACOES_FINAL)
    df_pares_enriquecidos = (df_pares_total
        .join(df_contas_local.alias("orig"), df_pares_total.id_conta_origem == F.col("orig.id"), "left")
        .join(df_contas_local.alias("dest"), df_pares_total.id_conta_destino == F.col("dest.id"), "left")
        .join(df_chaves_recentes_local.alias("chaves"), df_pares_total.id_conta_destino == F.col("chaves.id_conta"), "left")
        .select("id_conta_origem", "id_conta_destino", F.col("orig.id_tipo_conta").alias("id_tipo_conta_origem"),
                F.col("dest.id_tipo_conta").alias("id_tipo_conta_destino"), F.col("chaves.chave_destino_cadastrada_em"), F.col("dest.is_high_risk")))
    gerador_com_contexto = functools.partial(_gerar_detalhes_transacao_python_vetorizado, ano=ano, mes=mes, config=config_geracao, perfis_uso=perfis_de_uso_dict)
    df_transacoes_bruto = df_pares_enriquecidos.mapInPandas(gerador_com_contexto, schema=SCHEMA_TRANSACOES_UDF)
    return df_transacoes_bruto.withColumn("estado_ibge", F.lit(estado_ibge))

# --- Funções Utilitárias e de Orquestração ---
def salvar_dataframe_em_delta(df: DataFrame, nome_tabela_completo: str, modo: str = "append"):
    if df is None or df.isEmpty(): print(f"AVISO: DataFrame para a tabela '{nome_tabela_completo}' está vazio. Nenhuma ação tomada."); return
    try:
        print(f"INFO: Salvando dados na tabela Delta: {nome_tabela_completo} (modo: {modo})..."); df.write.format("delta").mode(modo).saveAsTable(nome_tabela_completo)
        print(f"✅ SUCESSO: Dados salvos em {nome_tabela_completo}.")
    except Exception as e: print(f"❌ ERRO ao salvar '{nome_tabela_completo}': {e}"); raise e

def gerar_e_salvar_populacao(num_pf: int, num_pj: int, estado_ibge: int, municipio_ibge: int) -> DataFrame:
    print(f"INFO: Gerando e materializando população para {municipio_ibge} (PF: {num_pf}, PJ: {num_pj})...")
    df_clientes_gerado = _gerar_clientes(num_pf, num_pj, estado_ibge, municipio_ibge); salvar_dataframe_em_delta(df_clientes_gerado, "transacoes_db.copper.clientes", modo="append")
    df_clientes_materializado = spark.table("transacoes_db.copper.clientes").filter(F.col("municipio_ibge") == municipio_ibge)
    df_contas_gerado = _gerar_contas(df_clientes_materializado, estado_ibge, municipio_ibge)
    salvar_dataframe_em_delta(df_contas_gerado, "transacoes_db.copper.contas", modo="append")
    df_contas_materializado = spark.table("transacoes_db.copper.contas").filter(F.col("municipio_ibge") == municipio_ibge)
    df_chaves_pix = _gerar_chaves_pix(df_contas_materializado, df_clientes_materializado, estado_ibge, municipio_ibge); salvar_dataframe_em_delta(df_chaves_pix, "transacoes_db.copper.chaves_pix", modo="append")
    print(f"INFO: População para o município {municipio_ibge} adicionada com sucesso.")
    return df_contas_materializado

def limpar_tabelas_de_destino():
    print("INFO: Apagando tabelas de destino para recriação...")
    for tabela in ["clientes", "contas", "chaves_pix", "transacoes"]: spark.sql(f"DROP TABLE IF EXISTS transacoes_db.copper.{tabela}")
    print("INFO: ✅ Limpeza concluída.")

def criar_tabelas_de_destino():
    print("INFO: Criando tabelas de destino com os schemas corretos...")
    tabelas = {"transacoes_db.copper.clientes": (SCHEMA_CLIENTES_FINAL, ["estado_ibge", "municipio_ibge"]),
               "transacoes_db.copper.contas": (SCHEMA_CONTAS_FINAL, ["estado_ibge", "municipio_ibge"]),
               "transacoes_db.copper.chaves_pix": (SCHEMA_CHAVES_PIX_FINAL, ["estado_ibge", "municipio_ibge"]),
               "transacoes_db.copper.transacoes": (SCHEMA_TRANSACOES_FINAL, ["estado_ibge"])}
    for nome, (schema, part_cols) in tabelas.items():
        spark.createDataFrame([], schema).write.format("delta").partitionBy(part_cols).mode("overwrite").saveAsTable(nome)
        print(f"INFO: Tabela '{nome}' criada com sucesso.")

# Modelagem Matemática para Simulação de Dados Sintéticos

---

## 1. Introdução

Este documento detalha a modelagem matemática e estatística utilizada para gerar uma população e um volume de transações sintéticas que refletem o comportamento e as características de um ambiente financeiro real, em conformidade com dados estatísticos regionais (perfis de uso) e regras de negócio.

A modelagem se divide em três etapas principais:

1. **Dimensionamento do volume**
2. **Dimensionamento da população**
3. **Modelagem comportamental para a geração de transações**

---

## 2. Dimensionamento do Volume de Transações

O volume total de transações a ser gerado para o município é baseado em dados estatísticos conhecidos (média mensal de transações) e ajustado ao escopo da simulação por meio de um fator de amostragem.

### 2.1. Fórmula do Volume Anual

O volume total anual de transações a gerar ($V_{\text{ano}}$) é calculado pela seguinte relação:

$$
V_{\text{ano}} = (\overline{T}_{\text{mensal}} \times 12) \times P_{\text{amostra}}
$$

| Termo                | Definição                                                                                  |
|----------------------|-------------------------------------------------------------------------------------------|
| $V_{\text{ano}}$     | Volume anual total de transações a ser gerado.                                            |
| $\overline{T}_{\text{mensal}}$ | Média estatística do número de transações de pagadores por mês no município.         |
| $P_{\text{amostra}}$ | Percentual da amostra desejada (`DATA_SAMPLE_PERCENTAGE`), fração do dado real a simular. |

---

## 3. Dimensionamento da População Sintética

Com o volume de transações definido, é necessário estimar o número de clientes que compõem a população sintética, garantindo que o volume transacional seja distribuído de forma crível.

### 3.1. Cálculo do Número Total de Clientes

O número total de clientes a serem gerados ($N_{\text{clientes}}$) é estimado a partir da média mensal de transações, ajustado por um fator de escala de população ($F_{\text{escala}}$) e o percentual da amostra:

$$
N_{\text{clientes}} = \left\lfloor \frac{\overline{T}_{\text{mensal}}}{F_{\text{escala}}} \times P_{\text{amostra}} \right\rfloor
$$

| Termo             | Definição                                                                                      |
|-------------------|-----------------------------------------------------------------------------------------------|
| $N_{\text{clientes}}$ | Número total de clientes (Pessoa Física e Jurídica) a gerar.                                  |
| $F_{\text{escala}}$   | Fator de escala para relacionar volume transacional com número de clientes (`POPULATION_SCALING_FACTOR`). |

### 3.2. Distribuição por Natureza (Pessoa Física e Jurídica)

O total de clientes é dividido em Pessoa Física ($N_{\text{PF}}$) e Pessoa Jurídica ($N_{\text{PJ}}$) com base na proporção observada de transações de Pessoa Física ($\%_{\text{PF}}$) no município:

$$
N_{\text{PF}} = \lfloor N_{\text{clientes}} \times \%_{\text{PF}} \rfloor
$$

$$
N_{\text{PJ}} = N_{\text{clientes}} - N_{\text{PF}}
$$

---

## 4. Modelagem Comportamental e Geração de Transações

Esta seção detalha os modelos estatísticos e as regras condicionais utilizadas para gerar as transações individuais (valores, datas e características de fraude).

### 4.1. Distribuição Proporcional por Perfil

Para garantir que a "mistura" de tipos de transação no município sintético reflita a região, o volume total é distribuído proporcionalmente aos perfis de uso definidos.

Primeiro, calcula-se o peso ($W_i$) de cada Perfil $i$:

$$
W_i = \frac{Q_i}{\sum_{j=1}^{P} Q_j}
$$

Onde $Q_i$ é a quantidade de transações do Perfil $i$ e $\sum_{j=1}^{P} Q_j$ é a soma das quantidades de todos os perfis ($P$).

Em seguida, calcula-se o volume de transações específicas para o Perfil $i$ ($V_i$):

$$
V_i = \lfloor V_{\text{ano}} \times W_i \rfloor
$$

---

### 4.2. Modelagem de Valores Monetários (Distribuição Log-Normal)

A **Distribuição Log-Normal** é empregada para modelar valores monetários (saldos e valores de transação) devido à sua assimetria e natureza não negativa, o que se alinha ao comportamento de dados financeiros.

Os parâmetros $\hat{\mu}$ (média do logaritmo) e $\hat{\sigma}$ (desvio padrão do logaritmo) são estimados a partir de estatísticas robustas (mediana e quartis) dos perfis para calibrar a geração:

$$
\hat{\mu} = \ln(\text{Mediana})
$$

$$
\hat{\sigma} = \frac{\ln(P_{75}) - \ln(P_{25})}{1.349}
$$

O valor final da transação ($X$) é gerado a partir desta distribuição, onde o valor base é multiplicado por fatores de ajuste para simular fraudes ou valores outliers.

---

### 4.3. Simulação de Eventos (Ensaio de Bernoulli e Lógica Condicional)

#### A. Ensaio de Bernoulli

A ocorrência de eventos discretos, como a marcação de uma transação como fraude, é modelada como um **Ensaio de Bernoulli**, onde a variável aleatória binária $X$ (sucesso/fracasso) é determinada por um número aleatório $U \sim \text{Uniforme}(0, 1)$ e uma probabilidade dinâmica $p$:

$$
X =
\begin{cases}
1, & \text{se } U < p \ (\text{sucesso/fraude}) \\
0, & \text{se } U \geq p \ (\text{falha/normal})
\end{cases}
$$

#### B. Lógica Condicional Vetorizada (`np.select`)

A probabilidade de fraude ($P_{\text{fraude}}$), que define o parâmetro $p$ do Ensaio de Bernoulli, é calculada em função de múltiplas regras de negócio e características da transação (vetorizada pelo `np.select`), como demonstrado no exemplo:

$$
P_{\text{fraude}} =
\begin{cases}
P_1, & \text{se a conta destino é de alto risco} \\
P_2, & \text{se } \Delta_{\text{dias}} \leq D_{\text{recente}} \text{ (chave recente)} \\
P_3, & \text{caso contrário}
\end{cases}
$$

---

### 4.4. Simulação de Cadeias de Lavagem (Distribuição de Dirichlet)

Em cenários de fraude complexa (e.g., triangulação), a **Distribuição de Dirichlet** é utilizada para simular a pulverização de um valor total em $N$ sub-transações aleatórias, garantindo que a soma das proporções seja igual a 1.

O vetor de proporções $\vec{x} = (x_1, \dots, x_N)$ é gerado com base em parâmetros de concentração $\alpha_i$ (tipicamente uniformes, $\alpha_i = 1$) e aplicado ao valor da transação raiz: $V_i = x_i \cdot V_{\text{raiz}}$

$$
(x_1, \dots, x_N) \sim \mathrm{Dirichlet}(\alpha_1, \dots, \alpha_N)
$$

$$
\sum_{i=1}^N x_i = 1
$$

O valor de cada sub-transação $V_i$ é dado por:

$$
V_i = x_i \cdot V_{\text{raiz}}
$$

---

## 5. Conclusão

A integração da **Distribuição Log-Normal**, do **Ensaio de Bernoulli**, da **Lógica Condicional Vetorizada** e da **Distribuição de Dirichlet** permite que o script de geração simule de forma eficaz um ecossistema financeiro robusto, com comportamentos e patamares de risco que se assemelham aos dados reais.

---

In [0]:
# =============================================================================
# CÉLULA 4: ORQUESTRAÇÃO E EXECUÇÃO PRINCIPAL (VERSÃO FINAL OTIMIZADA)
# =============================================================================
try:
    limpar_tabelas_de_destino()
    criar_tabelas_de_destino()
    print("=============================================================================")
    print(f"INFO: Iniciando processo de geração ANUAL (Ano: {ANO_ESTATISTICA})...")
    
    print("INFO: Lendo volumes anuais da tabela agregada...")
    df_volumes_anuais = spark.table("transacoes_db.pix_baseline_metricas.volumes_anuais_por_municipio").filter(F.col("Ano") == ANO_ESTATISTICA)
    
    print("INFO: Rankeando municípios para seleção...")
    df_ranks_anuais = df_volumes_anuais.withColumn("rank_pagador_anual", F.rank().over(Window.orderBy(F.col("volume_pagador_anual").desc())))
    
    municipios_a_processar_lista = df_ranks_anuais.orderBy(F.col("rank_pagador_anual").asc()).limit(LIMITE_MUNICIPIOS_PROCESSADOS).collect()
    total_municipios = len(municipios_a_processar_lista)
    print(f"INFO: {total_municipios} municípios selecionados para processar.")
    
    id_municipios_selecionados = [row["cod_ibge_municipio"] for row in municipios_a_processar_lista]

    print("INFO: Coletando estatísticas mensais para os municípios selecionados...")
    stats_mensal_pd = (spark.table("transacoes_db.pix_baseline_metricas.relacao_pagadores_recebedores")
                       .filter((F.col("ano") == ANO_ESTATISTICA) & (F.col("Municipio_Ibge").isin(id_municipios_selecionados)))
                       .select("Municipio_Ibge", "Mes", "total_tx_pf_pagador", "total_tx_pj_pagador").toPandas())
    stats_mensal_pd['total_tx_pagador'] = stats_mensal_pd['total_tx_pf_pagador'] + stats_mensal_pd['total_tx_pj_pagador']
    print("INFO: Estatísticas mensais coletadas com sucesso.")

    municipios_ja_processados = []

    for i, municipio_row in enumerate(municipios_a_processar_lista):
        codigo_municipio = municipio_row["cod_ibge_municipio"]
        codigo_estado = municipio_row["cod_ibge_estado"]
        nome_municipio = municipio_row["municipio_nome"]
        
        print(f"\n================== Processando Município {i+1}/{total_municipios}: {nome_municipio} ({codigo_municipio}) ==================")
        
        fator_escala_final = FATOR_ESCALA_VOLUME
        print(f"INFO: Fator de escala para {nome_municipio}: {fator_escala_final:.8f}")

        volume_pf_anual = int(municipio_row["total_pf_anual"] * fator_escala_final)
        volume_pj_anual = int(municipio_row["total_pj_anual"] * fator_escala_final)
        num_pf = max(1, int(volume_pf_anual / (TX_POR_CLIENTE_ESPERADO * 12)))
        num_pj = max(1, int(volume_pj_anual / (TX_POR_CLIENTE_ESPERADO * 12)))

        df_contas_do_municipio = gerar_e_salvar_populacao(num_pf=num_pf, num_pj=num_pj, estado_ibge=codigo_estado, municipio_ibge=codigo_municipio)
        municipios_ja_processados.append(codigo_municipio)

        # --- AJUSTE FINAL: PREPARAÇÃO DOS DADOS FEITA UMA VEZ POR MUNICÍPIO ---
        print("INFO: Preparando dados do município para o loop mensal (executado uma vez)...")
        num_contas_do_municipio = df_contas_do_municipio.count()
        
        df_chaves_do_municipio = spark.table("transacoes_db.copper.chaves_pix").filter(F.col("municipio_ibge") == codigo_municipio)
        window_chaves = Window.partitionBy("id_conta").orderBy(F.col("cadastrada_em").desc())
        df_chaves_recentes_do_municipio = (df_chaves_do_municipio.withColumn("rank", F.rank().over(window_chaves))
                                           .filter(F.col("rank") == 1).select("id_conta", F.col("cadastrada_em").alias("chave_destino_cadastrada_em")))

        for mes in range(1, 13):
            print(f"\n--- Processando Mês {mes}/{ANO_ESTATISTICA} para {nome_municipio} ---")
            
            stats_mensal = stats_mensal_pd[(stats_mensal_pd['Municipio_Ibge'] == codigo_municipio) & (stats_mensal_pd['Mes'] == mes)]
            if stats_mensal.empty: print(f"AVISO: Sem estatísticas para {mes}/{ANO_ESTATISTICA}. Pulando."); continue
            
            volume_total_original = stats_mensal["total_tx_pagador"].iloc[0]
            volume_total = int(volume_total_original * fator_escala_final)
            if volume_total <= 0: print(f"AVISO: Volume de transações base para {mes}/{ANO_ESTATISTICA} é 0 ou negativo. Pulando."); continue
                
            print(f"      Volume Original: {volume_total_original} | Volume BASE Alvo: {volume_total}")
            
            df_transacoes = gerar_transacoes(
                df_contas_local=df_contas_do_municipio,
                df_chaves_recentes_local=df_chaves_recentes_do_municipio,
                num_contas_local=num_contas_do_municipio,
                volume_total=volume_total, estado_ibge=codigo_estado, municipio_ibge=codigo_municipio,
                ano=ANO_ESTATISTICA, mes=mes, municipios_processados=municipios_ja_processados
            )
            salvar_dataframe_em_delta(df_transacoes, "transacoes_db.copper.transacoes", modo="append")
finally:
    print("\nINFO: O script chegou ao fim.")

print("\n=============================================================================")
print("INFO: Processo de geração de dados sintéticos concluído.")
print("=============================================================================")

In [0]:
%sql
CREATE OR REPLACE TABLE transacoes_db.gold.transacoes_dataset
SELECT
  -- Colunas da transação
  tx.id AS transacao_id,
  tx.valor AS valor_transacao,
  tx.data AS data_transacao,
  tx.mensagem AS mensagem_pix,
  tx.id_conta_origem AS id_conta_pagador,
  tx.id_conta_destino AS id_conta_recebedor,
  tx.id_tipo_iniciacao_pix AS tipo_iniciacao_pix_id,
  tx.id_finalidade_pix AS finalidade_pix_id,
  tx.is_fraud AS transacao_fraudulenta,
  tx.fraud_type AS tipo_fraude,
  tx.id_transacao_cadeia_pai AS id_transacao_cadeia_pai,
  tx.estado_ibge AS estado_ibge_transacao,

  -- Pagador: Conta
  conta_orig.id AS pagador_conta_id,
  conta_orig.saldo AS pagador_saldo,
  conta_orig.aberta_em AS pagador_conta_aberta_em,
  conta_orig.agencia AS pagador_agencia,
  conta_orig.numero AS pagador_numero_conta,
  conta_orig.id_tipo_conta AS pagador_tipo_conta_id,
  conta_orig.ispb_instituicao AS pagador_ispb_instituicao,
  conta_orig.id_cliente AS pagador_cliente_id_conta,
  conta_orig.is_high_risk AS pagador_conta_alto_risco,
  conta_orig.estado_ibge AS pagador_estado_ibge,
  conta_orig.municipio_ibge AS pagador_municipio_ibge,

  -- Pagador: Cliente
  cliente_orig.id AS pagador_cliente_id,
  cliente_orig.nome AS pagador_nome,
  cliente_orig.id_natureza AS pagador_natureza_id,
  cliente_orig.registro_nacional AS pagador_registro_nacional,
  cliente_orig.nascido_em AS pagador_data_nascimento,
  cliente_orig.estado_ibge AS pagador_estado_ibge_cliente,
  cliente_orig.municipio_ibge AS pagador_municipio_ibge_cliente,

  -- Pagador: Instituição
  inst_orig.ispb AS pagador_instituicao_ispb,
  inst_orig.nome AS pagador_instituicao,

  -- Pagador: Tipo de Conta
  tipo_conta_orig.id AS pagador_tipo_conta_id_ref,
  tipo_conta_orig.nome AS pagador_tipo_conta_descricao,

  -- Pagador: Município
  mun_orig.codigo_ibge AS pagador_municipio_ibge_ref,
  mun_orig.nome AS pagador_municipio,

  -- Pagador: Natureza
  natureza_orig.id AS pagador_natureza_id_ref,
  natureza_orig.nome AS pagador_natureza,

  -- Recebedor: Conta
  conta_dest.id AS recebedor_conta_id,
  conta_dest.saldo AS recebedor_saldo,
  conta_dest.aberta_em AS recebedor_conta_aberta_em,
  conta_dest.agencia AS recebedor_agencia,
  conta_dest.numero AS recebedor_numero_conta,
  conta_dest.id_tipo_conta AS recebedor_tipo_conta_id,
  conta_dest.ispb_instituicao AS recebedor_ispb_instituicao,
  conta_dest.id_cliente AS recebedor_cliente_id_conta,
  conta_dest.is_high_risk AS recebedor_conta_alto_risco,
  conta_dest.estado_ibge AS recebedor_estado_ibge,
  conta_dest.municipio_ibge AS recebedor_municipio_ibge,

  -- Recebedor: Cliente
  cliente_dest.id AS recebedor_cliente_id,
  cliente_dest.nome AS recebedor_nome,
  cliente_dest.id_natureza AS recebedor_natureza_id,
  cliente_dest.registro_nacional AS recebedor_registro_nacional,
  cliente_dest.nascido_em AS recebedor_data_nascimento,
  cliente_dest.estado_ibge AS recebedor_estado_ibge_cliente,
  cliente_dest.municipio_ibge AS recebedor_municipio_ibge_cliente,

  -- Recebedor: Instituição
  inst_dest.ispb AS recebedor_instituicao_ispb,
  inst_dest.nome AS recebedor_instituicao,


  -- Recebedor: Tipo de Conta
  tipo_conta_dest.id AS recebedor_tipo_conta_id_ref,
  tipo_conta_dest.nome AS recebedor_tipo_conta_descricao,

  -- Recebedor: Município
  mun_dest.codigo_ibge AS recebedor_municipio_ibge_ref,
  mun_dest.nome AS recebedor_municipio,

  -- Recebedor: Natureza
  natureza_dest.id AS recebedor_natureza_id_ref,
  natureza_dest.nome AS recebedor_natureza


FROM
  transacoes_db.copper.transacoes AS tx

LEFT JOIN transacoes_db.copper.contas AS conta_orig
  ON tx.id_conta_origem = conta_orig.id
LEFT JOIN transacoes_db.copper.clientes AS cliente_orig
  ON conta_orig.id_cliente = cliente_orig.id
LEFT JOIN transacoes_db.copper.instituicoes AS inst_orig
  ON conta_orig.ispb_instituicao = inst_orig.ispb
LEFT JOIN transacoes_db.copper.tipos_conta AS tipo_conta_orig
  ON conta_orig.id_tipo_conta = tipo_conta_orig.id
LEFT JOIN transacoes_db.copper.municipios AS mun_orig
  ON cliente_orig.municipio_ibge = mun_orig.codigo_ibge
LEFT JOIN transacoes_db.copper.naturezas AS natureza_orig
  ON cliente_orig.id_natureza = natureza_orig.id

LEFT JOIN transacoes_db.copper.contas AS conta_dest
  ON tx.id_conta_destino = conta_dest.id
LEFT JOIN transacoes_db.copper.clientes AS cliente_dest
  ON conta_dest.id_cliente = cliente_dest.id
LEFT JOIN transacoes_db.copper.instituicoes AS inst_dest
  ON conta_dest.ispb_instituicao = inst_dest.ispb
LEFT JOIN transacoes_db.copper.tipos_conta AS tipo_conta_dest
  ON conta_dest.id_tipo_conta = tipo_conta_dest.id
LEFT JOIN transacoes_db.copper.municipios AS mun_dest
  ON cliente_dest.municipio_ibge = mun_dest.codigo_ibge
LEFT JOIN transacoes_db.copper.naturezas AS natureza_dest
  ON cliente_dest.id_natureza = natureza_dest.id

LEFT JOIN transacoes_db.copper.finalidade_pix AS finalidade_pix
  ON tx.id_finalidade_pix = finalidade_pix.id

ORDER BY
  tx.data DESC